# ETL Project

Table 1: Qin, Bryan, Ryan, Seth

* Call Demograhic Data from Census API
* Load Election Data from CSV
* Clean Data and ensure join can be made on fips Code
* Load Data in a mysql DB

## Extract Data

In [ ]:
# election data csv can be found here:
#  https://github.com/tonmcg/US_County_Level_Election_Results_08-16

# Import Dependencies
# config file stored locally with necessary credentials
from census import Census
import pandas as pd
from sqlalchemy import create_engine
from config import cen_key, mysqlUser, mysqlPswd

# Create Census variable to be passed to API call
cen_config = Census(cen_key, year=2016)

# Get Census Data
censusData = cen_config.acs1.get(
                                    ( 
                                     "NAME", 
                                    "B19013_001E", #household income (median)
                                    "B01003_001E", #population (total)
                                    "B09001_001E", #population under 18
                                    "B01002_001E", #median age
                                    "B19301_001E", #per capita income (average income per person)
                                    "B17001_002E", #poverty count
                                    "B23025_005E", #unemployment count
                                    "B15012_001E", #total recorded bachelor degrees
                                    "B15003_002E", 
                                    "B15003_017E",
                                    "B15003_018E",
                                    "B15003_021E",
                                    "B15003_022E",
                                    "B15003_023E",
                                    "B15003_024E",
                                    "B15003_025E"
                                    ),
                                    {'for': 'county:*'} 
                                    ) 

# Create DataFrame out of returned Census Data
censusDf = pd.DataFrame(censusData)

# Create DataFrame out of Election Result CSV
file_path = './2016ElectionData.csv'
elecData = pd.read_csv(file_path, index_col=0)

## Transform Data

In [ ]:
# ------ Clean API Data ------
# Rename columns to be more descriptive
#  Use underscores instead of spaces to allow more
#  functionality in mySQL
re_censusDf = censusDf.rename(columns={
                                    "NAME": "Name", 
                                    "state": "State_ID",
                                    "B19013_001E": "Household_Income",
                                    "B01003_001E": "Population",
                                    "B09001_001E": "Population_Under_18",
                                    "B01002_001E": "Median_Age",
                                    "B19301_001E": "Per_Capita_Income",
                                    "B17001_002E": "Poverty_Count",
                                    "B23025_005E": "Unemployment_Count",
                                    "B15012_001E": "Total_Recorded_Bachelor_Degrees",
                                    "B15003_002E": "No_Education",
                                    "B15003_017E": "High_School",
                                    "B15003_018E": "GED",
                                    "B15003_021E": "Associates",
                                    "B15003_022E": "Bachelors",
                                    "B15003_023E": "Masters",
                                    "B15003_024E": "Professional_Degree",
                                    "B15003_025E": "Doctorate"
                                    })

# Create fips code by combining state and county ids
# drop state and county columns, no longer necessary
re_censusDf['Fips_Code'] = re_censusDf['State_ID'] + re_censusDf['county']
re_censusDf = re_censusDf.drop(['State_ID','county'],axis=1)

# reorder columns into cleaner format
cols = re_censusDf.columns.tolist()
cols = cols[-4:] + cols[:-4]
re_censusDf = re_censusDf[cols]


# ------ Clean CSV Data ------
# CSV had fips code as 4 digits instead of 5
# where fips code should start with 0
# Function to add 0 to the front of the fips code (fips code under 5 digis)
def math(x):
    if len(str(x)) < 5:
        return "0" + str(x)
    else:
        return str(x)
    
# Apply math function and create a new column
# Drop old Fips code
elecData['Fips_Code'] = elecData['combined_fips'].apply(math)
elecData = elecData.drop(['combined_fips'],axis=1)


# Alaska Data is repeating for all fips codes
# Cannot distinguish actual Data
# Remove all Alaska Data
elecData = elecData[elecData.state_abbr != "AK"]

## Load Data

In [ ]:
# mySQL schema needs to be created on your local machine prior to running code
# schema name is etl_project

# Create connection to mySQL database
rds_connection_string = f"{mysqlUser}:{mysqlPswd}@127.0.0.1/etl_project"
engine = create_engine(f'mysql://{rds_connection_string}')

# read dataframes into mysql tables. if they are already there, replace them
elecData.to_sql(name='election_data', con=engine, if_exists='replace')
re_censusDf.to_sql(name='census_data', con=engine, if_exists='replace')